# Introduction


- Chemistry
    - Experiments are expensive and not environmentally friendly
- Computational Chemistry:
    - Simulate and predict chemical reactions using computers to solve equations from quantum chemistry

### Why quantum computing?
- Richard Feynman:
    - "Nature isn’t classical, and if you want to make a simulation of Nature, you’d better make it quantum mechanical"
- Examples:
    - $N_2$ fixation reactions at less harsh conditions than Haber process (~1% of CO2 emitted)
    - Needs multireference methods, but the computational cost of these methods is beyond digital computers

## Quantum computing in chemistry:

### Noisy intermediate-scale quantum (NISQ) era
- Currently, hardware for quantum computing algorithms that can potentially outperform classical computational chemistry is not available
    - Quantum phase-estimation (QPE) algorithm:
        - In theory: we know the molecular Hamiltonian (see below) exactly
        - Hence, can simulate a chemical system directly using a quantum computer
    - But not possible in practical terms with current level of quantum hardware


### Variational Quantum Eigensolver (VQE) approach
- Based on variational theorem (quantum mechanics):
    - Exact energy < Approximate energy
- Define a loss function (electronic energy), and minimise it
    - Combines classical and quantum computing:
        - Classical: Optimizers and molecular integrals
        - Quantum: Represent the electronic wave function
    - 2 main approaches (ansatz):
        - Chemistry-based:
            - Expensive, need to make cheaper
        - Heuristic/Hardware-efficient:
            - Cheaper, but often not as reliable and/or efficient
- On the whole, circuit depth is significantly reduced compared to QPE

## Introduction to Quantum Chemistry

#### Schrodinger equation:

$$
\hat{H} \Psi = E \Psi
$$

For fixed set of nuclear coordinates, the electronic Hamiltonian $\hat{H}$ is

$$
\begin{equation}
\hat{H} = \hat{T}_e + \hat{V}_{nn} + \hat{V}_{ne} + \hat{V}_{ee}
\end{equation}
$$

$$
\begin{align}
\text{where } T &= \text{Kinetic energy} \\
V &= \text{Electrostatic interaction} \\
n, e &= \text{nuclei, electrons}
\end{align}
$$

- One of the main goals in computational quantum chemistry:
    - Solve for $\Psi$ = Electronic wave function
    - Allows us to know everything about our molecule
- Problem: Cannot be solved exactly because of electron-electron repulsion
- Solve approximately instead:
    - Simplify problem using a finite basis set:
        - For atoms: Atomic orbitals
            - Localised Gaussian-type functions (exp$(-ax^2)$) about each atom in a molecule
        - For molecules: Molecular orbitals
            - Linear combination of atomic orbitals (LCAO)
                - e.g. $H_2\text{: } \chi_1 = c_1 \phi_1 + c_2 \phi_2$ and $\chi_2 = c_1 \phi_1 - c_2 \phi_2$
    - Hartree-Fock method:
        - Solve for the LCAO coefficients s.t. the total molecular energy is minimized
        - Then 'fill up' the lowest $N_{\text{e}}$ (= number of electrons) molecular orbitals

#### Second quantization formalism
- Working in the space of molecular orbitals
- Fermionic annihilation/creation operators
    - $\hat{a}_p$/$\hat{a}_p^\dagger$: Annihilate/Create electron in molecular orbital $p$


Electronic Hamiltonian in this formalism:
$$
\hat{H} = \sum_{p, q} h_{pq} \hat{a}_p^\dagger \hat{a}_q
+ \frac{1}{2} \sum_{p, q, r, s} h_{pqrs} \hat{a}_p^\dagger \hat{a}_q^\dagger \hat{a}_r \hat{a}_s
$$

using the molecular integrals, $h_{pq}$ and $h_{pqrs}$

$$
h_{pq} = \int \chi_{p}^{*}(\mathbf{x}) \left( -\frac{\nabla^2}{2} - \sum_{A} \frac{Z_A}{\lvert \mathbf{r} - \mathbf{R}_A \rvert} \right) \chi_{q}(\mathbf{x}) d\mathbf{x}
$$

$$
h_{pqrs} = \int \frac{\chi_{p}^{*}(\mathbf{x}_1) \chi_{q}^{*}(\mathbf{x}_2) \chi_{r}(\mathbf{x}_2) \chi_{s}(\mathbf{x}_1)}{\lvert \mathbf{r}_1 - \mathbf{r}_2 \rvert} d\mathbf{x}_1 d\mathbf{x}_2
$$

The electronic wave function is then just

$$
a_{N_e}^\dagger a_{N_e -1}^\dagger \dots a_{2}^\dagger a_{1}^\dagger \lvert 0 \rangle
$$

where $\lvert 0 \rangle$ is the vacuum state, i.e. a state with no particles

In [ ]:
# QChem: Hands-on
# Part 1. Setting up and transforming the Chemistry problem to quantum computing

# Import required modules
import numpy as np

# "OpenFermion is a library for compiling and analyzing quantum algorithms to simulate fermionic systems, 
# including quantum chemistry"
import openfermion

# Molecule class to store the molecular integrals and other relevant quantities
from qibochem.driver.molecule import Molecule

In [ ]:
# The molecule of interest can be defined using an .xyz file
h2 = Molecule(xyz_file='h2.xyz')

# Then, run a classical computational chemistry software to obtain the molecular integrals

# From their respective manuals:
# "Psi4 is an open-source suite of ab initio quantum chemistry programs"
h2.run_psi4()
# h2.run_pyscf() # Probably can't get PySCF on a Windows system so easily...

# The one- (oei) and two- (tei) electron integrals are obtained after running the chemistry driver
print("One- electron integrals for H2:")
print(h2.oei)
print()
print("Two- electron integrals for H2:")
print(h2.tei)

In [ ]:
# With these integrals, we can form the molecular Hamiltonian (see above) as a sum of creation/annihilation operators
fermionic_hamiltonian = h2.fermionic_hamiltonian() # OpenFermion InteractionOperator
# Convert to an OpenFermion FermionOperator
print(openfermion.transforms.get_fermion_operator(fermionic_hamiltonian))

## Quantum computing applied to Chemistry


### Fermion-to-Qubit Transformation

Lastly, the Chemistry (fermionic) problem has to be mapped onto a (qubit) problem that can be modelled using a quantum computer

E.g. Jordan-Wigner transformation:

$$\text{Annihilation, } \hat{a}_p \rightarrow \hat{Z}^{\otimes p} \otimes \frac{1}{2} (\hat{X} + i\hat{Y}) \otimes 1^{\otimes N_{qubits} -p-1}$$

$$\text{Creation, } \hat{a}_p^\dagger \rightarrow \hat{Z}^{\otimes p} \otimes \frac{1}{2} (\hat{X} - i\hat{Y}) \otimes 1^{\otimes N_{qubits} -i-1}$$

Note: In this mapping, each molecular orbital has a one-to-one mapping to a qubit, i.e. the $Z$-measurement of a qubit gives the occupancy of its associated molecular orbital directly.

In [ ]:
# Apply the Jordan-Wigner transformation to the molecular Hamiltonian
jw_mol_hamiltonian = openfermion.jordan_wigner(fermionic_hamiltonian)
print(jw_mol_hamiltonian)

### Measurements to calculate the energy expectation value

- To get the energy of our $H_2$ molecule:
    - Need to measure the terms in the above cell, such as $0.1771\dots Z_0$ and $0.1705\dots Z_0 Z_1$
    - For X/Y measurements:
        - Basis rotation gates have to be applied before measuring the output of the circuit
- Because of errors in running the quantum circuit:
    - Measurement of each Pauli string has to be repeated numerous times
- Expectation value of a Pauli string:
    - Average value over all measurements
- Total molecular energy:
    - Sum of expectation values of all Pauli strings defining the qubit molecular Hamiltonian

In [ ]:
from qibo import models, gates

from qibochem.measurement.expectation import pauli_expectation_shots, circuit_expectation_shots

In [ ]:
# Example: the 0.1777 Z0 term for H2

# First, we build a basic quantum circuit with 4 (= number of spin-orbitals, nso) qubits
circuit = models.Circuit(h2.nso)
print(circuit.draw())

In the Jordan-Wigner mapping, each qubit represents the occupancy of a molecular spin-orbital.
The empty circuit with no gates is then simply just the vacuum state.
Despite that, we can still obtain the expectation value of a Pauli string for this simple circuit, as shown below.

In [ ]:
circuit = models.Circuit(h2.nso)
circuit.add(gates.M(0)) # Add a measurement gate to measure Z
print(circuit.draw())

# Define the Z0 QubitOperator  
z0_qubit_operator = openfermion.QubitOperator('Z0', 0.1777)
print(f"\nTerm: {z0_qubit_operator}\n")

# The pauli_expectation_shots(circuit, nshots) helper function runs and measures the output of circuit nshots times
# Since we're looking at the Z0 term, there's no need to apply any basis rotation gates before measurement
z0_expectation_value = pauli_expectation_shots(circuit, 100)
print(f"Expectation value: {z0_expectation_value}")

Note: pauli_expectation_shots just gives the measurement directly, which is just equal to 1.0 for a circuit with no gates applied

In [ ]:
# The circuit_expectation_shots(circuit, QubitOperator, nshots) helper function can be used to get the expectation value
# of a QubitOperator, including its coefficient, directly

circuit = models.Circuit(h2.nso)

circuit_expectation = circuit_expectation_shots(circuit, z0_qubit_operator, 100)
print(f"Expectation value: {circuit_expectation}")

### Simulated quantum circuits

- Since our simulated quantum circuit doesn't have any noise, the expectation value of the Pauli string is exactly 0.1777.
- Furthermore, since we're using a simulated quantum circuit here, we can obtain the exact state of the 'qubit wave function' for the circuit
    - The state vector of the circuit can be used with the qubit Hamiltonian directly to calculate the expectation value of a Pauli string, and by extension, the qubit molecular Hamiltonian

In [ ]:
# The same 0.1777 Z0 term, using the circuit state vector:
# Note: We need to broadcast the SymbolicHamiltonian using I (I as in identity) terms to match 4 qubits
# We use qibo.symbols here because openfermion.QubitOperator doesn't have I
from qibo.symbols import I, Z
from qibo.hamiltonians import SymbolicHamiltonian

# The broadcasted Z_0 term using qibo.symbol
z0_qubit_operator_broadcasted = 0.1777*Z(0)*I(1)*I(2)*I(3)
# Convert it to a Qibo SymbolicHamiltonian
z0_symbolic = SymbolicHamiltonian(z0_qubit_operator_broadcasted)

# Build and run the simulated circuit to obtain its state vector
circuit = models.Circuit(h2.nso)
circuit_result = circuit(nshots=1)
circuit_state = circuit_result.state()
# Calculate the expectation value using the state vector
z0_expectation_value = z0_symbolic.expectation(circuit_state)

print(f"Expectation value: {z0_expectation_value}")

In [ ]:
# Note: Helper function in Molecule class for expectation values
# TODO: Might change in next version of Qibo (0.1.12)

# Using the broadcasted SymbolicHamiltonian
circuit = models.Circuit(h2.nso)
print(f"Expectation value: {h2.expectation_value(circuit, z0_symbolic)}")

Going forward, we will just use the expectation value of the molecular Hamiltonian, obtained using the simulated state vector, for convenience.

In [ ]:
# Example: Electronic energy of the vacuum state
circuit = models.Circuit(h2.nso)

# Get the molecular Hamiltonian, and convert it to a SymbolicHamiltonian
fermion_hamiltonian = h2.fermionic_hamiltonian()
qubit_hamiltonian = h2.qubit_hamiltonian(fermion_hamiltonian)
symbolic_hamiltonian = h2.symbolic_hamiltonian(qubit_hamiltonian)

print(f" Electronic energy of vacuum state: {h2.expectation_value(circuit, symbolic_hamiltonian)}")

# Since there are no electrons, the electronic energy just consists of the nuclear repulsion energy
print(f"(Same as) Nuclear repulsion energy: {h2.e_nuc}")

## VQE: Ansatz representing the molecular wave function

We have shown above how to calculate the molecular electronic energy from a quantum circuit.
The next step is to use a parametrized quantum circuit to model the electronic wave function of the molecule, i.e. the circuit ansatz.
The aim of VQE is thus to minimise the electronic energy with respect to the parameters of the circuit ansatz.

Here, we will use the [hardware-efficient ansatz](https://www.nature.com/articles/nature23879) to represent the wave function of the $H_2$ molecule.
Note: it's actually possible to use symmetry to further reduce the number of qubits needed to just 2, but we will stick to using the one-to-one Jordan-Wigner mapping of a spin-orbital to a qubit directly for clarity in this example)

In [ ]:
from qibochem.ansatz.hardware_efficient import hea_su2

In [ ]:
# Get the list of quantum gates representing the ansatz
gate_list = hea_su2(nlayers=1, nqubits=h2.nso) # 1 layer is sufficient

# Build a quantum circuit and add the gates from gate_list to it
circuit = models.Circuit(h2.nso)
circuit.add(gate for gate in gate_list)
print(circuit.draw())

In [ ]:
# Get the number of paramerized gates
n_param_gates = len(circuit.get_parameters())

# Create the VQE model
vqe = models.VQE(circuit, symbolic_hamiltonian)

# Optimize starting from a random guess for the variational parameters
initial_parameters = np.random.uniform(0, 2*np.pi,
                                        n_param_gates)
best, params, extra = vqe.minimize(initial_parameters)# method='BFGS', compile=False) 

In [ ]:
print("Results:")
print(f"{' ':>9}VQE energy: {best}\n")

print(f"Classical HF energy: {h2.e_hf}")
# Exact groundstate energy of H2 as a reference
print(f"{' ':>5}Exact solution: {symbolic_hamiltonian.eigenvalues()[0]}")

Notes:
- The first reference result, the classical HF energy, is the basic starting point for most classical computational chemistry calculations.
    - To outperform classical computational chemistry, the energy from the quantum circuit should be lower than this value
    - However, depending on the initial set of parameters and the classical optimizer, one might occasionally reach this result as a local minimum
- Of course, ideally, the VQE process manages to find the second reference result, the exact solution from diagonalizing the molecular Hamiltonian exactly